<a href="https://colab.research.google.com/github/coderdevanshpro/AIML-project/blob/main/Devansh_Sharma_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.linear_model import LinearRegression

# Sample data
x = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])

# Create and train model
model = LinearRegression()
model.fit(x, y)

# Predict
y_pred = model.predict([[6]])
print("Prediction for x=6:", y_pred)


Prediction for x=6: [12.]


In [1]:
pip install rembg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.3 MB/s eta 0:00:00


In [2]:
pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


In [3]:
from rembg import remove
from PIL import Image

# Load image
input_path = "person.jpeg"
output_path = "person_no_bg.png"

# Remove background
input_image = Image.open(input_path)
output_image = remove(input_image)
output_image.save(output_path)


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 178GB/s]


In [4]:
import cv2
import numpy as np

# Load background and person
background = cv2.imread("scenery.jpg")
person = cv2.imread("person_no_bg.png", cv2.IMREAD_UNCHANGED)

# Get background size
bh, bw = background.shape[:2]

# User input for scale
scale_percent = int(input("Enter the % of background height the person should cover: "))
target_height = int(bh * (scale_percent / 100.0))

# Resize person
scale_factor = target_height / person.shape[0]
person = cv2.resize(person, (int(person.shape[1] * scale_factor), target_height))

# Get person dimensions
ph, pw = person.shape[:2]

# Position person (bottom-center)
x_offset = (bw - pw) // 2
y_offset = bh - ph - 10

# Extract alpha channel
if person.shape[2] == 4:
    alpha = person[:, :, 3] / 255.0
    alpha_inv = 1 - alpha
    person_rgb = person[:, :, :3]
else:
    alpha = np.ones((ph, pw))
    alpha_inv = 1 - alpha
    person_rgb = person

# Blend person into background
roi = background[y_offset:y_offset+ph, x_offset:x_offset+pw]
for c in range(3):
    roi[:, :, c] = alpha * person_rgb[:, :, c] + alpha_inv * roi[:, :, c]
background[y_offset:y_offset+ph, x_offset:x_offset+pw] = roi

cv2.imwrite("final_output.png", background)
print("Saved final_output.png")


Enter the % of background height the person should cover: 90
Saved final_output.png


In [5]:
# ======================================================
# INSTALL REQUIRED LIBRARIES
# ======================================================
!pip install --quiet mediapipe opencv-python-headless pillow numpy

# ======================================================
# IMPORTS
# ======================================================
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance
import os

# ======================================================
# FUNCTION TO ADD SHADOW
# ======================================================
def add_shadow_to_person(
    image_path,
    output_path="image_with_shadow.png",
    angle_deg=30,       # direction of shadow (0=right, 90=down)
    shadow_distance=40, # distance from person
    shadow_opacity=100, # 0-255 alpha
    blur_radius=15      # softness of shadow edges
):
    # Load image
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        raise FileNotFoundError(f"Could not load image: {image_path}")
    h, w = img_bgr.shape[:2]

    # PERSON SEGMENTATION
    mp_selfie = mp.solutions.selfie_segmentation.SelfieSegmentation(model_selection=1)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    result = mp_selfie.process(img_rgb)
    person_mask = (result.segmentation_mask > 0.5).astype(np.uint8) * 255

    # Convert BGR image to RGBA
    img_rgba = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2BGRA)
    img_rgba[:, :, 3] = person_mask  # alpha channel = person mask

    # Convert to PIL for easier shadow processing
    person_pil = Image.fromarray(cv2.cvtColor(img_rgba, cv2.COLOR_BGRA2RGBA))

    # Create shadow mask from alpha
    alpha = person_pil.split()[3]

    # Create shadow layer
    shadow = Image.new("RGBA", person_pil.size, (0, 0, 0, 0))
    shadow.paste((0, 0, 0, shadow_opacity), mask=alpha)
    shadow = shadow.filter(ImageFilter.GaussianBlur(blur_radius))

    # Shift shadow
    dx = int(np.cos(np.radians(angle_deg)) * shadow_distance)
    dy = int(np.sin(np.radians(angle_deg)) * shadow_distance)
    shadow = shadow.transform(
        person_pil.size, Image.AFFINE, (1, 0, dx, 0, 1, dy), resample=Image.BICUBIC
    )

    # Composite shadow with original image
    base_img = Image.open(image_path).convert("RGBA")
    final_img = Image.alpha_composite(base_img, shadow)
    final_img = Image.alpha_composite(final_img, person_pil)

    # Save
    final_img.save(output_path)
    print(f"Saved: {output_path}")
    return output_path

# ======================================================
# USAGE EXAMPLE
# ======================================================
input_image = "final_output.png"  # your image path
output_image = "image_with_shadow.png"



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [6]:

add_shadow_to_person(
    input_image,
    output_image,
    angle_deg=200,       # 200° = left-down direction
    shadow_distance=35,  # distance of shadow
    shadow_opacity=150,  # darkness
    blur_radius=1       # softness
)

Saved: image_with_shadow.png


'image_with_shadow.png'